In [19]:
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout
from keras.layers import BatchNormalization, Activation
from keras.initializers import VarianceScaling
from keras.optimizers import Adam
from keras.utils import np_utils, multi_gpu_model
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping
np.set_printoptions(suppress=True)

k = 0
trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
print(len(trainL))
for i in range(len(trainL)):
    if(trainL[i] != 1):
        trainL[i] = 0
        k = k+1
    else:
        print(i)
print(len(trainL))
print(k)

7110
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
7110
6900


In [2]:
trainD[0]

array([0.519, 0.619, 0.723, ..., 0.116, 0.017, 0.018])

In [3]:
from biosppy.signals import ecg
olpeaks = []
for i in trainD:
    peaks = ecg.christov_segmenter(i, 300)
    olpeaks.append(len(peaks[0]))
olpeaks = np.array(olpeaks)

In [24]:
from sklearn.metrics import accuracy_score
print(olpeaks[:451])
print('----------------------')
print(olpeaks[452:601])
print(olpeaks[5259:5318])
for j in range(30):
    r_Label = []
    for i in olpeaks:
        if i <= j:
            r_Label.append(1)
        else:
            r_Label.append(0)
    r_Label = np.array(r_Label)
    score = accuracy_score(trainL, r_Label)
    print(score)

[32 57 51 45 40 44  2 40 48 51 37 52 29 55 30 44 44 39 38 36 48  1 59 43
 41 36 43 44 47 48 44 43 58 59 45 45 54 10 40 16 44 55 38 37 36 35 55 12
 30 34 37 40 39 71 57 38 35 32 51 52 40 56  3 72 71 56 43 36 41 52 57 44
 59 37 33 31 39 34 21 41 43 35 54 37 66 66 37 54 57 29 29 60 59 50 64 40
 40 37 53 68 41 32 64 35 32 35 34 34 49 47 48 37 36 37 55 40 50 52 37 43
 37 58 36 34 38 42 36 53 52 63 66 42 38 45 33 49 73 48 58 33 38 46 43 38
 49 41 49 34  2 58 54 35 34 40 35 42 37 69 66 75 31 47 43 45 74 44 57 58
 34 34 59 50 74 74 60 44 33 66 32 54 54 30 36 32 26 50 50  6  4 29 73 50
 49 38 46 43  1 35 11 35 36 37 68 43 40 46 57 52 45 47 35 42 57 33 44 45
 70 31 33 50 38 32 42 29 30 49 42 61 63 61 31 39 33 50 34 48 47 30 44 45
 29 33 40 19 28 36 46 44 18 26 75 63 38 60 35 39 74 47 55 33 40 43 64 36
  1 46 59 66 49 42 50 37 40 55 52 30 35 39 26 68 67 66 31 66 64 35 39 27
 62 33 43 51 47 44 53 53 13 40 54 43 35 43 38 40 44 29 34 68 45 44 30 30
 36 30 42 56 21 26 41 53 59 28 52 46 50 39 46 50 70